_[Please follow the instructions [here](https://github.com/Hermi999/bce) to see how to properly use this bash-kernel Notebook]_

# Beyond Corp Enterprise - Demo

The following notebook is used to guide you through setting up Beyond Corp Enterprise (BCE) on Google Cloud Platform (GCP). 

## Context
As more employees work from home, it is important to provide more options for them to securely access internal services and sites.
These sites may be your intranet portal, sensitive HR or financial systems that are not exposed directly externally. Traditionally, companies accessed these sites after establishing a VPN tunnel from an employee’s laptop directly to the corporate network. Just about any administrator of such a system will tell you VPNs are expensive to procure, require a lot of resources, quite cumbersome to maintain and essentially provide security based on dynamically expanding the trusted network.

At Google, we found the access model based primarily on a network perimeter did not sufficiently scale with a mobile workforce nor provide selective security guarantees for sensitive applications. Our approach was to assess the overall context of a given request for an application. By that we would take signals derived from multiple sources at runtime to evaluate the integrity of a call against the resource being accessed. For example, the evaluation signals for a request would include who is making the call, from what device, from what network, at what time, and so on. A request is not simply evaluated using an administrator defined static if-then chain, but also assigned trust levels based on heuristics inferred from prior contexts.

The internal system Google used to achieve this is described in six BeyondCorp research papers which goes into more details about these signals. For our customers we make this same system available as __BeyondCorp Enterprise (BCE).__

As you can see on the image below, BCE consists of multiple components:
- Cloud Identity
- Identity Aware proxy
- Google Cloud Load Balancer
- Access Context Manager
- IAP Connector

![BCE](./img/bce.png)

## How to use this colab notebook

*   If you didn't start at following Github Repository, please start there to see how you can use this bash-kernel based notebook: https://github.com/Hermi999/bce
*   **You can use this notebook either as a reference and copy the commands directly into GCP Cloud Shell, or you use it with a Jupyter Notebook with a bash kernel.**
*   If you like to execute a block use STRG+ENTER
*   If you just want to run highlighted commands use STRG+SHIFT+ENTER

## Getting ready
Let's try it out by checking the version of the gcloud sdk. Click into the next line and press STRG+ENTER

In [ ]:
gcloud version

We do not need to authenticate ourselfs to run gcloud commands. It automatically uses the Compute Engine Service Account which was copied onto the VM during creation.

You can check this by running the command:

In [ ]:
gcloud auth list

This Service Account should have all necessary rights to proceed.

We also double check if we are in the right project:

In [ ]:
gcloud projects list

If the project is wrong you can set your project with 'gcloud config set project [project_name].

## Deploy a first sample application
We start with a first sample application on GCP. 
For this we will use the __App Engine__, which was the first GCP service (2008) and is a PaaS for hosting web applications. 

Our Git repository we cloned onto this machine contains an App Engine sample code. 

In [ ]:
cd ~/bce/samples/appengine
ls

Now we can deploy the simple sample to App Engine:

In [ ]:
gcloud app create --region=europe-west3

Wait until you see the message gcloud app browse

In [ ]:
gcloud app deploy --quiet

In [ ]:
gcloud app browse

When you follow the link you should see an empty website with a simple sign in link.
Now that we have our app running it's time to secure it with Identity-aware proxy.

## Configure OAuth 
### The Consent Screen (=Brand)
(https://cloud.google.com/iap/docs/programmatic-oauth-clients)

Because IAP accesses Google users’ data (email address) you need to configure the OAuth.
This needs to be done once per GCP Project. Typically you manage multiple on-prem or AWS/Azure web apps from the same projects, so it only need to be done rearily. 

The OAuth consent screen, which contains branding information for users, is known as a brand. Brands can be limited to internal users or public users. An __internal brand__ makes the OAuth flow accessible to someone who belongs to the same Google Workspace organization as the project. A __public brand__ makes the OAuth flow available to anyone on the internet. There is only one brand per GCP project.

OAuth API verification is a little bit difficult to understand - read more about it here: https://support.google.com/cloud/answer/9110914

Let's create our brand:

In [ ]:
APP_TITLE=IAP_TEST
SUPPORT_EMAIL=hewagner@google.com

gcloud alpha iap oauth-brands create --application_title=$APP_TITLE --support_email=$SUPPORT_EMAIL

The support email displayed on the OAuth consent screen. This email address can either be a user's address or a Google Groups alias.

Now we can list our new brand and extract the Brand-ID:

In [ ]:
gcloud alpha iap oauth-brands list
BRAND=$(gcloud alpha iap oauth-brands list --format="value(name)")

echo "BRAND: $BRAND"

### The OAuth Client
(https://cloud.google.com/iap/docs/programmatic-oauth-clients#creating_an_oauth_client)

Now we create our OAuth Client for the IAP. You configure one OAuth client per app.

In [ ]:
CLIENT_NAME=iap
PROJECT_ID=$(gcloud config get-value core/project)

gcloud alpha iap oauth-clients create $BRAND --display_name=$CLIENT_NAME
#gcloud alpha iap oauth-clients create projects/$PROJECT_ID/brands/1023990209690 --display_name=NAME

Be aware that it's only possible to create oauth clients via gcloud if the brand is __internal__. Otherwise you have to use the GUI.

Now we enable IAP for the web service. Fill in the client_id and secret from the previous step:

In [ ]:
CLIENT_ID=
SECRET=
gcloud alpha iap web enable --resource-type=app-engine --oauth2-client-id=$CLIENT_ID --oauth2-client-secret=$SECRET

Now we have activated IAP for the App Engine App. This means that we do not have access any more...

In [ ]:
gcloud app browse

## Creating access levels
https://cloud.google.com/access-context-manager/docs/overview

While Identity-Aware Proxy (IAP) lets you manage who has access to your web services, with __Access Context Manager__ you can define __when__ (in which circumstances) the request is allowed. Access Context Manager allows Google Cloud organization administrators to define fine-grained, attribute based access control for projects and resources in Google Cloud.

Administrators first define an __access policy__, which is an organization-wide container for __access levels__.
Access levels describe the necessary requirements for requests to be honored. Examples include:
- Device type and operating system
- IP address
- User identity
- Device Type (you need a BeyondCorp Enterprise or Cloud Identity Premium license)

### Use GitOps for Access Levels
We could create access policies and levels via the GCP Console UI, but that's too easy :) Instead we want to set up a GitOps-based approach for our access levels. This makes it easy to track changes and approve changes which can be submitted via a __merge request__.

But first let's have a look at the example policies I've prepared for you:

## Grant access on the application
The last step is to grant access to the protected app. 
We can do this on an individual user or group level and grant access to just a single app or to all apps in the whole project. First let's add yourself:

In [ ]:
USER="h@hewagner.com"
gcloud iap web add-iam-policy-binding --member="user:$USER" --role='roles/iap.httpsResourceAccessor'

Wait about a minute and try to access the web app url:

In [ ]:
gcloud app browse

You should see a welcome message and your email address.

## GCP Console
Of course you can also create and view everything we just did in a GUI (GCP Console). It should look similar to this:
![alt](img/iap.png)

Here are the links to have a look for yourself:

In [ ]:
PROJECT_ID=$(gcloud config get-value core/project)
ORG_ID=$(gcloud projects describe $PROJECT_ID --format="value(parent.id)")
echo "https://console.cloud.google.com/security/iap?project=$PROJECT_ID"
echo "https://console.cloud.google.com/security/access-level?orgonly=true&organizationId=$ORG_ID"